In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [151]:
dataset_path = '../DATASET/INITIAL_DATASET'
train_path = os.path.join(dataset_path, 'train.csv')
test_path = os.path.join(dataset_path, 'test.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [152]:
train_df['source'] = 'train'
test_df['source'] = 'test'

full_df = pd.concat([train_df, test_df], ignore_index=True)

In [153]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [154]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [155]:
result = pd.concat([train_df.isna().sum(),train_df.isna().mean()],axis=1)
result

,0,1
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,0.198653
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [156]:
df = full_df.copy()
le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

In [157]:
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Embarked'] = df['Embarked'].replace(3, np.nan)
ohetransform = ohe.fit_transform(df[['Sex']])
df = pd.concat([df, ohetransform], axis=1).drop(columns=['Sex'])

In [158]:
features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Embarked', 'Sex_male', 'Sex_female']
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df[features]), columns=features)

In [159]:
df_imputed['Embarked'] = df_imputed['Embarked'].round().astype(int)
df_imputed['Embarked'] = le.inverse_transform(df_imputed['Embarked'])

In [160]:
full_df['Embarked'] = full_df['Embarked'].fillna(df_imputed['Embarked'])


In [161]:
full_df = pd.concat([full_df, ohetransform], axis=1).drop(columns=['Sex'])

In [166]:
ohetransform = ohe.fit_transform(full_df[['Embarked']])
full_df = pd.concat([full_df, ohetransform], axis=1).drop(columns=['Embarked'])

In [167]:
full_df.dtypes

PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
source          object
Sex_female     float64
Sex_male       float64
Embarked_C     float64
Embarked_Q     float64
Embarked_S     float64
dtype: object

In [168]:
full_df['Fare'] = full_df['Fare'].fillna(df_imputed['Fare'])

In [169]:
full_df['Age_missing'] = full_df['Age'].isna().astype(int)

In [170]:
full_df['Age'] = full_df['Age'].fillna(df_imputed['Age'])

In [174]:
train_df = full_df[full_df['source']=='train'].copy().drop(columns=['source', 'Name', 'Ticket', 'Cabin'])
test_df = full_df[full_df['source']=='test'].copy().drop(columns=['source', 'Name', 'Ticket', 'Cabin'])


In [199]:
train_df.columns
features = ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Age_missing']
X_train = train_df[features]
y_train = train_df['Survived']

X_test = test_df[features]

In [188]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [192]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [203]:
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer


search_space = {
    'max_depth': Integer(2, 15),
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.3, 1.0)
}

# Create XGBoost classifier
xgb = XGBClassifier(n_estimators=200, objective='binary:logistic', random_state=42)

# Perform Bayesian optimization
bayes_search = BayesSearchCV(estimator=xgb, search_spaces=search_space, n_iter=250, cv=10, n_jobs=-1, verbose=2)
bayes_search.fit(X_train, y_train)

# Print best parameters
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best score: {bayes_search.best_score_}")
# test_predictions = model.predict(X_test)
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': test_predictions
# })
# submission.to_csv('titanic_xgb_bayes_submission.csv', index=False)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.2s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.2s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.2s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.2s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.1s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, max_depth=9, subsample=0.7981204473525189; total time=   0.1s
[CV] END colsample_bytree=0.7755314342685985, learning_rate=0.06031802983111176, 

In [204]:
# Predict on the test set using the best found model
test_predictions = bayes_search.predict(X_test)

# Prepare the submission file
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})

submission.to_csv('titanic_xgb_bayes_submission_long.csv', index=False)